# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
import pkg_resources
import azureml.core
from azureml.widgets import RunDetails
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.datastore import Datastore
from azureml.pipeline.steps import AutoMLStep
from azureml.core.model import Model, InferenceConfig
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.core import Environment
from azureml.core.webservice import AciWebservice
import requests
import json

print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


In [2]:
pip show azureml-widgets

Name: azureml-widgets
Version: 1.51.0
Summary: Provides fully supported, with interactivity, async auto-updates, and non-blocking cell execution.
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: jinja2, azureml-core, ipywidgets, azure-storage-blob, azureml-telemetry
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
myenv = Environment(name = "myenv")

In [5]:
ws = Workspace.from_config()
myenv.register(workspace=ws)

# choose a name for experiment
experiment_name = 'maternal-health-automl-exp'

experiment=Experiment(ws, experiment_name)

In [6]:
dataset = Dataset.get_by_name(ws, name='maternal-health-risk')
df = dataset.to_pandas_dataframe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here
automl_settings = { "experiment_timeout_minutes": 30,
                    "max_concurrent_iterations": 5,
                    "primary_metric" : 'accuracy'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             task = "classification",
                             training_data=df,
                             label_column_name="Outcome",  
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [8]:
# TODO: Submit your experiment
from azureml.train.automl.run import AutoMLRun
remote_run = AutoMLRun(experiment = experiment, run_id = 'AutoML_9d4ad407-d089-41e7-85f1-fc88df4ab866')

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
print(remote_run.get_file_names())

['definition.json', 'definition_original.json', 'outputs/_automl_internal/_CACHE_STORE_KEY_ONNX_CONVERTER_INIT_METADATA_.pkl', 'outputs/_automl_internal/indices/861553f469944289acaa5cf0d54c7d94.json', 'outputs/verifier_results.json']


In [10]:
print(azureml.core.VERSION)

1.51.0


In [3]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

NameError: name 'remote_run' is not defined

In [12]:
run_details = remote_run.get_details()
print(run_details)

{'runId': 'AutoML_9d4ad407-d089-41e7-85f1-fc88df4ab866', 'target': 'notebook240923', 'status': 'Completed', 'startTimeUtc': '2023-09-01T14:55:27.517319Z', 'endTimeUtc': '2023-09-01T15:21:51.717023Z', 'services': {}, 'warnings': [{'source': 'JasmineService', 'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}], 'properties': {'num_iterations': '1000', 'training_type': 'TrainFull', 'acquisition_function': 'EI', 'primary_metric': 'AUC_weighted', 'train_split': '0', 'acquisition_parameter': '0', 'num_cross_validation': None, 'target': 'notebook240923', 'AMLSettingsJsonString': '{"enable_early_stopping":true,"enable_ensembling":true,"enable_stack_ensembling":true,"ensemble_iterations":15,"enable_onnx_compatible_models":false,"save_mlflow":true,"max_cores_per_iteration":-1,"send_telemetry":true,"blacklist_algos":["TensorFlowLin

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
auto_job, best_auto_model = remote_run.get_output()
print(auto_job)
print(best_auto_model)

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Run(Experiment: maternal-health-automl-exp,
Id: AutoML_9d4ad407-d089-41e7-85f1-fc88df4ab866_44,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/moun...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('14', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None, criterion='gini', max_depth=None, max_features='auto', max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=No

In [14]:
import joblib

# Save the best model
joblib.dump(value=best_auto_model, filename='auto_model.joblib')

['auto_model.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [20]:
model = auto_job.register_model(model_name='best-automl-model',
                                      model_path='outputs/model.pkl',
                                      description='Best model from AutoML for maternal health risk')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [21]:
auto_job.download_files(output_directory='automl_output')

env = Environment.from_conda_specification('automl-env', 'conda_dependencies.yml')

inference = InferenceConfig(environment=env, entry_script='./Score2.py')

deployment = AciWebservice.deploy_configuration(cpu_cores=1,
                                                       memory_gb=1,
                                                       enable_app_insights=True)

deployment_name = 'auto-maternal-risk'
service = Model.deploy(workspace=ws,
                       name=deployment_name,
                       models=[model],
                       inference_config=inference,
                       deployment_config=deployment,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-09-01 15:32:47+00:00 Creating Container Registry if not exists.
2023-09-01 15:32:47+00:00 Registering the environment.
2023-09-01 15:32:48+00:00 Building image..
2023-09-01 15:43:14+00:00 Generating deployment configuration.
2023-09-01 15:43:15+00:00 Submitting deployment to compute..
2023-09-01 15:43:20+00:00 Checking the status of deployment auto-maternal-risk..
2023-09-01 15:45:16+00:00 Checking the status of inference endpoint auto-maternal-risk.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [8]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data =  {
  "data": [
    {
      "Age": 0,
      "SystolicBP": 0,
      "DiastolicBP": 0,
      "BS": 15.0,
      "BodyTemp": 0,
      "HeartRate": 0
    }
  ]
}

body = str.encode(json.dumps(data))

url = 'http://17d79f0f-0926-4a48-b0a7-5e054aaa6758.westus2.azurecontainer.io/score'


headers = {'Content-Type':'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'"{\\"result\\": [1]}"'


In [24]:
logs = service.get_logs()
for line in logs.split('\n'):
    print(line)

/bin/bash: /azureml-envs/azureml_d23c40ccd7225ebe3e23cbbc514bfe04/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_d23c40ccd7225ebe3e23cbbc514bfe04/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_d23c40ccd7225ebe3e23cbbc514bfe04/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-09-01T15:45:07,541968100+00:00 - rsyslog/run 
2023-09-01T15:45:07,551178400+00:00 - gunicorn/run 
2023-09-01T15:45:07,552636700+00:00 | gunicorn/run | 
bash: /azureml-envs/azureml_d23c40ccd7225ebe3e23cbbc514bfe04/lib/libtinfo.so.6: no version information available (required by bash)
2023-09-01T15:45:07,560477600+00:00 | gunicorn/run | ###############################################
2023-09-01T15:45:07,562534500+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-09-01T15:45:07,568880100+00:00 | gunicorn/run | ########################################

In [ ]:
# Delete the Webservice and Compute Cluster
service.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
